In [ ]:
# http://www.imooc.com/article/43784

In [3]:
cd tencent_algo_2020/

/home/sayhi/workspaces/tencent_algo_2020


In [32]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

import time
import data
import lightgbm as lgb
import numpy as np
import os
import sys
import re
import pandas as pd
import matplotlib
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import mean_squared_error, roc_auc_score, accuracy_score
from gensim.models import word2vec, keyedvectors
import logging

import torch
import torch.nn as nn
import torchvision
from torchnet import meter
from torch.utils.data import Dataset, DataLoader
import torch.nn.utils.rnn as rnn_utils
# from mytorch.dataset import RecordDataset
# from mytorch.lstm import ClickRNN

from model import lgb_model


%matplotlib inline
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
'''
creative_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/creative_model.w2v", binary=True)
ad_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/ad_model.w2v", binary=True)
product_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/product_model.w2v", binary=True)
advertiser_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/advertiser_model.w2v", binary=True)
industry_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/industry_model.w2v", binary=True)
'''

'\ncreative_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/creative_model.w2v", binary=True)\nad_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/ad_model.w2v", binary=True)\nproduct_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/product_model.w2v", binary=True)\nadvertiser_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/advertiser_model.w2v", binary=True)\nindustry_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/industry_model.w2v", binary=True)\n'

In [6]:
train_ad, train_click, train_user, test_ad, test_click = data.load_data()
# train_record
train_record = pd.merge(train_click, train_ad, on="creative_id")
# test_record
test_record = pd.merge(test_click, test_ad, on="creative_id")

In [7]:
train_grouped = train_record.groupby("user_id")
test_grouped = test_record.groupby("user_id")

In [8]:
# sample data
sample_record = train_record[:5000]
sample_record

sample_grouped = sample_record.groupby("user_id")

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673.0,3,32638,319.0
1,15,320815,567330,1,504423,30673.0,3,32638,319.0
2,11,355089,567330,1,504423,30673.0,3,32638,319.0
3,9,363442,567330,1,504423,30673.0,3,32638,319.0
4,14,370513,567330,1,504423,30673.0,3,32638,319.0
...,...,...,...,...,...,...,...,...,...
4995,31,606288,1370845,1,1197877,26858.0,3,7170,60.0
4996,41,608609,1370845,1,1197877,26858.0,3,7170,60.0
4997,29,608609,1370845,1,1197877,26858.0,3,7170,60.0
4998,34,608749,1370845,1,1197877,26858.0,3,7170,60.0


In [14]:
list_grouped = sample_grouped.agg(list).reset_index()
list_grouped = pd.merge(list_grouped, train_user, on="user_id")
# for user_id, record in sample_grouped:

In [15]:
list_grouped
len(list_grouped)

,user_id,time,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,age,gender
0,8,[49],[2361327],[1],[2035918],[1261.0],[2],[6783],[6.0],5,1
1,90,[67],[3072255],[1],[2642300],[1261.0],[2],[6783],[6.0],3,2
2,117,[66],[2361327],[1],[2035918],[1261.0],[2],[6783],[6.0],2,1
3,217,[50],[2361327],[1],[2035918],[1261.0],[2],[6783],[6.0],7,2
4,528,"[71, 73, 49]","[3072255, 3072255, 2361327]","[1, 1, 1]","[2642300, 2642300, 2035918]","[1261.0, 1261.0, 1261.0]","[2, 2, 2]","[6783, 6783, 6783]","[6.0, 6.0, 6.0]",2,1
...,...,...,...,...,...,...,...,...,...,...,...
3872,899189,[89],[2851451],[1],[2451406],[29664.0],[3],[28212],[242.0],5,1
3873,899379,[72],[3072255],[1],[2642300],[1261.0],[2],[6783],[6.0],3,1
3874,899662,[61],[2361327],[1],[2035918],[1261.0],[2],[6783],[6.0],6,1
3875,899825,[6],[197866],[1],[177626],[26858.0],[3],[2367],[60.0],3,1


3877

In [9]:
class RecordDataset(Dataset):
    def __init__(self, list_grouped):
        self.creative_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/creative_model.w2v", binary=True)
        self.ad_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/ad_model.w2v", binary=True)
        self.product_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/product_model.w2v", binary=True)
        self.advertiser_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/advertiser_model.w2v", binary=True)
        self.industry_model = keyedvectors.KeyedVectors.load_word2vec_format("checkpoints/industry_model.w2v", binary=True)
        
        self.list_grouped = list_grouped
    
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        #这里需要注意的是，第一步：read one data，是一个data
        record = self.list_grouped.iloc[index, :]
        user_id = record["user_id"]
        t = len(record["ad_id"])
        age = record["age"]
        gender = record["gender"]
        
        # ad_embedding
        ad_embedding = self.get_embedding_from_grouped(user_id, record, column_name="ad_id")
        #creative_embedding
        creative_embedding = self.get_embedding_from_grouped(user_id, record, column_name="creative_id")
        #product_embedding
        product_embedding = self.get_embedding_from_grouped(user_id, record, column_name="product_id")
        #advertiser_embedding
        advertiser_embedding = self.get_embedding_from_grouped(user_id, record, column_name="advertiser_id")
        #industry_embedding
        industry_embedding = self.get_embedding_from_grouped(user_id, record, column_name="industry")
        
        embed_features = torch.cat([ad_embedding, creative_embedding, product_embedding, advertiser_embedding, industry_embedding], dim=1)
            
        return t, embed_features, age, gender
        
    def __len__(self):
        return len(self.list_grouped)
    
    
    def get_embedding_from_grouped(self, user_id, record, column_name):
        if column_name == "ad_id":
            model = self.ad_model
        elif column_name == "creative_id":
            model = self.creative_model
        elif column_name == "industry":
            model = self.industry_model
        elif column_name == "product_id":
            model = self.product_model
        elif column_name == "advertiser_id":
            model = self.advertiser_model

        if column_name == "industry":
            embedding = [np.zeros(100, ) if pd.isnull(x) else model[str(int(x))] for x in record[column_name]]
        elif column_name == "product_id":
            embedding = [np.zeros(200, ) if pd.isnull(x) else model[str(int(x))] for x in record[column_name]]
        else:
            embedding = [model[str(x)] for x in record[column_name]]
        
        embedding = torch.Tensor(embedding)
        
        return embedding


In [ ]:
sample_dataset = RecordDataset(list_grouped)

In [10]:
def collate_fn(data):
    data.sort(key = lambda x: x[0], reverse=True)
    t = [x[0] for x in data]
    embed_features = [x[1] for x in data]
    age = [x[2] for x in data]
    gender = [x[3] for x in data]

    embed_features = rnn_utils.pad_sequence(embed_features, batch_first=True, padding_value=0)
    # embed_features = rnn_utils.pack_padded_sequence(embed_features, lengths=t, batch_first=True)
    return t, embed_features, age, gender

In [ ]:
sample_dataloader = DataLoader(sample_dataset, batch_size=64, shuffle=True, collate_fn=collate_fn)

In [ ]:
t, embed_features, age, gender = iter(sample_dataloader).next()

In [12]:
class ClickRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, batch_size, bidirectional=True, drop_prob=0.5, train_on_gpu=False):
        super(ClickRNN, self).__init__()
         
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.bidirectional = bidirectional
        self.batch_size = batch_size
        self.train_on_gpu = train_on_gpu
        
        # self.embedding = nn.Embedding(input_dim, embedding_dim)

        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True,
                            bidirectional=bidirectional)
        
        # dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        # linear and sigmoid layers
        if bidirectional:
          self.fc = nn.Linear(hidden_dim*2, output_dim)
        else:
          self.fc = nn.Linear(hidden_dim, output_dim)
          
        self.sigmoid = nn.Sigmoid()
        
 
    def forward(self, x, hidden=None):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        
        # embeddings and lstm_out
#         x = x.long()
#         embeds = self.embedding(x)
        
        if hidden is None:
            hidden = self.init_hidden(self.train_on_gpu)
        
        lstm_out, hidden = self.lstm(x, hidden)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # sigmoid function
        sig_out = self.sigmoid(out)
        out = sig_out[:, -1, :]
        
        # return last sigmoid output and hidden state
        return out, hidden
    
    
    def init_hidden(self, train_on_gpu):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        number = 1
        if self.bidirectional:
           number = 2
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers*number, self.batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers*number, self.batch_size, self.hidden_dim).zero_().cuda()
                     )
        else:
            hidden = (weight.new(self.n_layers*number, self.batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers*number, self.batch_size, self.hidden_dim).zero_()
                     )
        
        return hidden

In [ ]:
gender_net = ClickRNN(input_dim=800, hidden_dim=200, output_dim=1, n_layers=1, batch_size=64)
age_net = ClickRNN(input_dim=800, hidden_dim=200, output_dim=10, n_layers=1, batch_size=64)

In [ ]:
gender_out, length, hidden = gender_net(embed_features)

In [ ]:
age_out, length, _ = age_net(embed_features)

In [33]:
def main(train_list, valid_list, model_type="gender"):
    batch_size = 64
    hidden_dim = 200
    train_on_gpu=True
    device = torch.device("cuda") if train_on_gpu else torch.device("cpu")
    
    model_path = None
    lr = 1e-5
    lr_decay = 0.95
    weight_decay = 1e-5
    
    max_epoch = 50
    
    train_dataset = RecordDataset(train_list)
    valid_dataset = RecordDataset(valid_list)
#     test_dataset = RecordDataset(test_list)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                              num_workers=8, drop_last=True, collate_fn=collate_fn)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, 
                              num_workers=8, drop_last=True, collate_fn=collate_fn)
#     test_loader = DataLoader(test_loader, batch_size=batch_size, shuffle=True, 
#                              num_workers=8, drop_last=True, collate_fn=collate_fn)
    
    print(len(train_dataset), len(valid_dataset))
    
    if model_type == "gender":
        model = ClickRNN(input_dim=800, hidden_dim=hidden_dim, output_dim=1, 
                         n_layers=2, batch_size=batch_size, train_on_gpu=train_on_gpu)
    elif model_type == "age":
        model = ClickRNN(input_dim=800, hidden_dim=hidden_dim, output_dim=10, 
                         n_layers=2, batch_size=batch_size, train_on_gpu=train_on_gpu)
        

    optimizer = torch.optim.Adam(
        params=model.parameters(),
        lr=lr,
        weight_decay=weight_decay,
        betas=(0.9, 0.99)
    )
    
    if model_path != None:
        map_location = lambda storage, loc: storage
        checkpoint = torch.load(model_path, map_location=map_location)
        model.load_state_dict(checkpoint["model"])
        last_epoch = checkpoint["epoch"]
        lr = checkpoint["lr"]
        optimizer.load_state_dict(checkpoint["optimizer"])

    model = model.to(device)
    
    last_epoch = -1
    previous_loss = 1000
    best_test_acc = 0
    best_model = None
    
    for epoch in range(max_epoch):
        if epoch < last_epoch:
            continue
            
        # ========================================START TRAIN============================================
        # train
        model.train()
        if model_type == "gender":
            criterion = nn.BCEWithLogitsLoss()
        elif model_type == "age":
            criterion = nn.CrossEntropyLoss()

        loss_meter = meter.AverageValueMeter()
        acc_meter = meter.AverageValueMeter()
        loss_meter.reset()
        acc_meter.reset()
        
        for ii, (lengths, embed_features, age, gender) in tqdm(enumerate(train_loader)):
            # input: [batch, time_step, input_size]
            if model_type == "gender":
                label = torch.Tensor(gender) - 1
            elif model_type == "age":
                label = torch.Tensor(age) - 1
                
            embed_features = embed_features.to(device)
            label = label.to(device)
            
            out, length, _ = model(embed_features)
            print(out.shape, label.shape)
            
            loss = criterion(out, label)
            loss.backward()

            optimizer.step()
            optimizer.zero_grad()
        
            acc = accuracy_score(label.cpu().data, out.cpu().data)
            acc_meter.add(acc)
            loss_meter.add(loss.item())
            
            print("epoch:{epoch}, lr:{lr:.5f}, train_loss:{loss:.5f}, train_acc:{acc:.5f}".format(epoch = epoch,
                                                                                                lr = lr,
                                                                                                loss = loss_meter.value()[0],
                                                                                                acc = acc_meter.value()[0]))
        # ========================================STOP TRAIN============================================
        # ========================================START VALID============================================
        model.eval()
        if model_type == "gender":
            criterion = nn.BCEWithLogitsLoss()
        elif model_type == "age":
            criterion = nn.CrossEntropyLoss()

        val_loss_meter = meter.AverageValueMeter()
        val_acc_meter = meter.AverageValueMeter()
        val_loss_meter.reset()
        val_acc_meter.reset()
        
        for ii, (lengths, embed_features, age, gender) in tqdm(enumerate(valid_loader)):
            # input: [batch, time_step, input_size]
            if model_type == "gender":
                label = torch.Tensor(gender) - 1
            elif model_type == "age":
                label = torch.Tensor(age) - 1
                
            embed_features = embed_features.to(device)
            label = label.to(device)
            
            out, length, _ = model(embed_features)
            print(out.shape, label.shape)
            
            loss = criterion(out, label)

            acc = accuracy_score(label.cpu().data, out.cpu().data)
            val_acc_meter.add(acc)
            val_loss_meter.add(loss.item())
            
            print("epoch:{epoch}, lr:{lr:.5f}, val_loss:{loss:.5f}, val_acc:{acc:.5f}".format(epoch = epoch,
                                                                                                lr = lr,
                                                                                                loss = val_loss_meter.value()[0],
                                                                                                acc = val_acc_meter.value()[0]))
        # ========================================STOP VALID============================================
        
        
        best_test_acc = max(best_test_acc, val_acc_meter.value()[0])
        
        if best_test_acc > val_acc_meter.value()[0]:
            best_test_acc = val_acc_meter.value()[0]
            best_model = model
        print("best_test_auc(val) is: ", best_test_acc)
        
        current_loss = loss_meter.value()[0]    
        print("current_loss: ", current_loss)
        if (current_loss > previous_loss) or ((epoch + 1) % 3) == 0:
            lr = lr * lr_decay
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr
        
        previous_loss = current_loss
        
        # TODO 每save_every个epoch结束后保存模型参数+optimizer参数
        if (epoch + 1) % 5 == 0:
            prefix = "checkpoints/LSTM_epoch{}_".format(epoch+1)
            file_name = time.strftime(prefix + '%m%d_%H_%M_%S.pth')
            checkpoint = {
                "epoch": epoch,
                "optimizer": optimizer.state_dict(),
                "model": model.state_dict(),
                "lr": lr
            }
            torch.save(checkpoint, file_name)
            
    # TODO 结束的时候保存final模型参数
    file_name = time.strftime('checkpoints/LSTM_final_%m%d_%H_%M_%S.pth')
    checkpoint = {
        "epoch": epoch,
        "optimizer": optimizer.state_dict(),
        "model": model.state_dict(),
        "lr": lr
    }
    torch.save(checkpoint, file_name)
    return best_model

In [ ]:
# train_list = train_grouped.agg(list).reset_index()
# train_list = pd.merge(train_list, train_user, on="user_id")

In [16]:
train_list, valid_list = train_test_split(list_grouped, test_size=0.33, random_state=42)

In [18]:
model_type = "age"
batch_size = 64
hidden_dim = 200
train_on_gpu=True
device = torch.device("cuda") if train_on_gpu else torch.device("cpu")

model_path = None
lr = 1e-5
lr_decay = 0.95
weight_decay = 1e-5

max_epoch = 50

train_dataset = RecordDataset(train_list)
valid_dataset = RecordDataset(valid_list)
#     test_dataset = RecordDataset(test_list)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                          num_workers=8, drop_last=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, 
                          num_workers=8, drop_last=True, collate_fn=collate_fn)
#     test_loader = DataLoader(test_loader, batch_size=batch_size, shuffle=True, 
#                              num_workers=8, drop_last=True, collate_fn=collate_fn)

print(len(train_dataset), len(valid_dataset))

if model_type == "gender":
    model = ClickRNN(input_dim=800, hidden_dim=hidden_dim, output_dim=1, 
                     n_layers=2, batch_size=batch_size, train_on_gpu=train_on_gpu)
elif model_type == "age":
    model = ClickRNN(input_dim=800, hidden_dim=hidden_dim, output_dim=10, 
                     n_layers=2, batch_size=batch_size, train_on_gpu=train_on_gpu)


optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=lr,
    weight_decay=weight_decay,
    betas=(0.9, 0.99)
)

if model_path != None:
    map_location = lambda storage, loc: storage
    checkpoint = torch.load(model_path, map_location=map_location)
    model.load_state_dict(checkpoint["model"])
    last_epoch = checkpoint["epoch"]
    lr = checkpoint["lr"]
    optimizer.load_state_dict(checkpoint["optimizer"])

model = model.to(device)

last_epoch = -1
previous_loss = 1000
best_test_acc = 0
best_model = None


2020-06-07 21:26:24,151 : INFO : loading projection weights from checkpoints/creative_model.w2v
2020-06-07 21:26:43,085 : INFO : loaded (3412772, 200) matrix from checkpoints/creative_model.w2v
2020-06-07 21:26:43,088 : INFO : loading projection weights from checkpoints/ad_model.w2v
2020-06-07 21:27:01,130 : INFO : loaded (3027360, 200) matrix from checkpoints/ad_model.w2v
2020-06-07 21:27:01,131 : INFO : loading projection weights from checkpoints/product_model.w2v
2020-06-07 21:27:01,320 : INFO : loaded (39056, 200) matrix from checkpoints/product_model.w2v
2020-06-07 21:27:01,321 : INFO : loading projection weights from checkpoints/advertiser_model.w2v
2020-06-07 21:27:01,587 : INFO : loaded (57870, 100) matrix from checkpoints/advertiser_model.w2v
2020-06-07 21:27:01,588 : INFO : loading projection weights from checkpoints/industry_model.w2v
2020-06-07 21:27:01,591 : INFO : loaded (331, 100) matrix from checkpoints/industry_model.w2v
2020-06-07 21:27:01,593 : INFO : loading project

2597 1280


In [34]:
def transform_pred(pred, model_kind):
#     pred = pred.numpy()
    if model_kind == "gender":
        record_pred_label = pred.copy()
        record_pred_label[pred >= 0.5] = 1
        record_pred_label[pred < 0.5] = 0
        record_pred_label = record_pred_label.astype(int)
    elif model_kind == "age":
        record_pred_label = [list(x).index(max(x)) for x in pred]

    return record_pred_label

In [35]:
for epoch in range(max_epoch):
    if epoch < last_epoch:
        continue

    # ========================================START TRAIN============================================
    # train
    model.train()
    if model_type == "gender":
        criterion = nn.BCEWithLogitsLoss()
    elif model_type == "age":
        criterion = nn.CrossEntropyLoss()

    loss_meter = meter.AverageValueMeter()
    acc_meter = meter.AverageValueMeter()
    loss_meter.reset()
    acc_meter.reset()

    for ii, (lengths, embed_features, age, gender) in tqdm(enumerate(train_loader)):
        # input: [batch, time_step, input_size]
        if model_type == "gender":
            label = torch.Tensor(gender) - 1
        elif model_type == "age":
            label = torch.Tensor(age) - 1

        embed_features = embed_features.to(device)
        label = label.long()
        label = label.to(device)

        out, _ = model(embed_features)

        loss = criterion(out, label)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        acc = accuracy_score(label.cpu().data, transform_pred(out.cpu().data, model_type))
        acc_meter.add(acc)
        loss_meter.add(loss.item())

    print("epoch:{epoch}, lr:{lr:.5f}, train_loss:{loss:.5f}, train_acc:{acc:.5f}".format(epoch = epoch,
                                                                                        lr = lr,
                                                                                        loss = loss_meter.value()[0],
                                                                                        acc = acc_meter.value()[0]))
    # ========================================STOP TRAIN============================================
    # ========================================START VALID============================================
    model.eval()
    if model_type == "gender":
        criterion = nn.BCEWithLogitsLoss()
    elif model_type == "age":
        criterion = nn.CrossEntropyLoss()

    val_loss_meter = meter.AverageValueMeter()
    val_acc_meter = meter.AverageValueMeter()
    val_loss_meter.reset()
    val_acc_meter.reset()

    for ii, (lengths, embed_features, age, gender) in tqdm(enumerate(valid_loader)):
        # input: [batch, time_step, input_size]
        if model_type == "gender":
            label = torch.Tensor(gender) - 1
        elif model_type == "age":
            label = torch.Tensor(age) - 1

        embed_features = embed_features.to(device)
        label = label.long()
        label = label.to(device)

        out, _ = model(embed_features)
        
        loss = criterion(out, label)
        
        acc = accuracy_score(label.cpu().data, transform_pred(out.cpu().data, model_type))
        val_acc_meter.add(acc)
        val_loss_meter.add(loss.item())

    print("epoch:{epoch}, lr:{lr:.5f}, val_loss:{loss:.5f}, val_acc:{acc:.5f}".format(epoch = epoch,
                                                                                        lr = lr,
                                                                                        loss = val_loss_meter.value()[0],
                                                                                        acc = val_acc_meter.value()[0]))
    # ========================================STOP VALID============================================


    best_test_acc = max(best_test_acc, val_acc_meter.value()[0])

    if best_test_acc > val_acc_meter.value()[0]:
        best_test_acc = val_acc_meter.value()[0]
        best_model = model
    print("best_test_auc(val) is: ", best_test_acc)

    current_loss = loss_meter.value()[0]    
    print("current_loss: ", current_loss)
    if (current_loss > previous_loss) or ((epoch + 1) % 5) == 0:
        lr = lr * lr_decay
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

    previous_loss = current_loss

    # TODO 每save_every个epoch结束后保存模型参数+optimizer参数
    if (epoch + 1) % 10 == 0:
        prefix = "checkpoints/LSTM_epoch{}_".format(epoch+1)
        file_name = time.strftime(prefix + '%m%d_%H_%M_%S.pth')
        checkpoint = {
            "epoch": epoch,
            "optimizer": optimizer.state_dict(),
            "model": model.state_dict(),
            "lr": lr
        }
        torch.save(checkpoint, file_name)

# TODO 结束的时候保存final模型参数
file_name = time.strftime('checkpoints/LSTM_final_%m%d_%H_%M_%S.pth')
checkpoint = {
    "epoch": epoch,
    "optimizer": optimizer.state_dict(),
    "model": model.state_dict(),
    "lr": lr
}
torch.save(checkpoint, file_name)

ClickRNN(
  (lstm): LSTM(800, 200, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=400, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

40it [00:01, 38.12it/s]

epoch:0, lr:0.00001, train_loss:2.24323, train_acc:0.20781


ClickRNN(
  (lstm): LSTM(800, 200, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=400, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

20it [00:00, 47.69it/s]

...

20it [00:00, 39.40it/s]


epoch:14, lr:0.00001, val_loss:2.11088, val_acc:0.20547
best_test_auc(val) is:  0.20546875
current_loss:  2.1157576143741608


ClickRNN(
  (lstm): LSTM(800, 200, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=400, out_features=10, bias=True)
  (sigmoid): Sigmoid()
)

Exception ignored in: <function _releaseLock at 0x7ff4958d0c20>
Traceback (most recent call last):
  File "/home/sayhi/anaconda3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
a, b = model(embed_features)

In [23]:
c = nn.CrossEntropyLoss()
c(out, label)

tensor(2.3003, device='cuda:0', grad_fn=<NllLossBackward>)

In [ ]:
def get_embedding_from_grouped(user_id, records, column_name, keep_uid=False):
    if column_name == "ad_id":
        model = ad_model
    elif column_name == "creative_id":
        model = creative_model
    elif column_name == "industry":
        model = industry_model
    elif column_name == "product_id":
        model = product_model
    elif column_name == "advertiser_id":
        model = advertiser_model
    
    if column_name == "industry":
        embedding = records[column_name].apply(lambda x: np.zeros(100, ) if pd.isnull(x) else model[str(int(x))]).apply(pd.Series)
    elif column_name == "product_id":
        embedding = records[column_name].apply(lambda x: np.zeros(200, ) if pd.isnull(x) else model[str(int(x))]).apply(pd.Series)
    else:
        embedding = records[column_name].apply(lambda x: model[str(x)]).apply(pd.Series)
    print(embedding.shape)
#     embedding = embedding.mean()
    
    if keep_uid:
        embedding.insert(0, "user_id", user_id)
    return embedding


flag = 0
for user_id, records in sample_grouped:
    records = records.sort_values(by="time")
    
    # ad_embedding
    ad_embedding = get_embedding_from_grouped(user_id, records, column_name="ad_id")
    #creative_embedding
    creative_embedding = get_embedding_from_grouped(user_id, records, column_name="creative_id")
    #product_embedding
    product_embedding = get_embedding_from_grouped(user_id, records, column_name="product_id")
    #advertiser_embedding
    advertiser_embedding = get_embedding_from_grouped(user_id, records, column_name="advertiser_id")
    #industry_embedding
    industry_embedding = get_embedding_from_grouped(user_id, records, column_name="industry")

    embed_features = np.concatenate([ad_embedding, creative_embedding, product_embedding, advertiser_embedding, industry_embedding], axis=1)
    print(embed_features.shape)
    print(user_id)
#     print(records)
    records
    
    flag += 1
    if flag >= 10:
        break

In [ ]:
def total_embed(grouped, data_type="sample"):
    flag = 0
    if data_type == "sample":
        f = open("embed/train/sample_embedding.csv", "w")
    elif data_type == "train":
        f = open("embed/train/train_embedding_800_{}.csv".format(id), "w")
    else:
        f = open("embed/test/n_test_embedding_800_{}.csv".format(id), "w")
    for user_id, records in tqdm(grouped):
        records = records.sort_values(by="time")

        # ad_embedding
        ad_embedding = get_embedding_from_grouped(user_id, records, column_name="ad_id")
        #creative_embedding
        creative_embedding = get_embedding_from_grouped(user_id, records, column_name="creative_id")
        #product_embedding
        product_embedding = get_embedding_from_grouped(user_id, records, column_name="product_id")
        #advertiser_embedding
        advertiser_embedding = get_embedding_from_grouped(user_id, records, column_name="advertiser_id")
        #industry_embedding
        industry_embedding = get_embedding_from_grouped(user_id, records, column_name="industry")

        embed_features = np.concatenate([ad_embedding, creative_embedding, product_embedding, advertiser_embedding, industry_embedding])
        f.write(str(user_id) + ', ' + str(list(embed_features))[1:-1] + '\n')
        '''
        左开右闭，下标从0开始
        0: userid
        [1:201]: ad_embedding
        [201:401]: creative_embedding
        [401:601]: product_embedding
        [601:701]: advertiser_embedding
        [701:801]: industry_embedding
        '''
        
#         flag += 1
#         if flag % 45000 == 0:
#             f.close()
#             id += 1
#             if data_type == "train":
#                 f = open("embed/train/train_embedding{}.csv".format(id), "w")
#             else:
#                 f = open("embed/test/test_embedding{}.csv".format(id), "w")
    f.close()

In [ ]:
total_embed(sample_grouped, data_type="sample")

In [ ]:
def place_zero(value):
    matched = value.group()
    return matched[0] + ", " + matched[2]

In [ ]:
root_path = "embed/test/"

files = (os.listdir(root_path))
if ".ipynb_checkpoints" in files:
    files.remove(".ipynb_checkpoints")
if "test_embedding_all.csv" in files:
    files.remove("test_embedding_all.csv")
files = [root_path + i for i in files]

with open(root_path + "test_embedding_all.csv", "w") as ff:
    for ii, file in enumerate(files):
        print(ii)
        f = open(file, "r")
        persons = f.readlines()
        for p in tqdm(persons):
            p1 = re.sub(r'\d\s\d', place_zero, p)
            t = ff.write(p1)
        f.close()

In [ ]:
a = re.search(r'\d\s-', p)
print(len(a.group()))

In [ ]:
# root_path = "embed/train/"
# i_file = root_path + "train_embedding_all.csv"
# o_file = root_path + "train_embedding_all_1.csv"

root_path = "embed/test/"
i_file = root_path + "test_embedding_all.csv"
o_file = root_path + "test_embedding_all_1.csv"

with open(o_file, "w") as ff:
    f = open(i_file, "r")
    persons = f.readlines()
    for p in tqdm(persons):
        if re.search(r'\d\s-', p) != None:
            p1 = re.sub(r'\d\s-', place_zero, p)
            t = ff.write(p1)
        else:
            t = ff.write(p)
    f.close()

In [ ]:
column_names = ["creative_id", "ad_id", "product_id", "advertiser_id", "industry"]
w2v_models = [creative_model, ad_model, product_model, advertiser_model, industry_model]

def transform_dataframe(train_features, column_names, w2v_models):
    for column_name, w2v_model in zip(column_names, w2v_models):
        print(column_name, "START")
        if column_name == "industry":
            embedding_df = train_features[column_name].apply(lambda x: np.zeros(100, ) if pd.isnull(x) else w2v_model[str(int(x))]).apply(pd.Series)
        elif column_name == "product_id":
            embedding_df = train_features[column_name].apply(lambda x: np.zeros(200, ) if pd.isnull(x) else w2v_model[str(int(x))]).apply(pd.Series)
        else:
            embedding_df = train_features[column_name].apply(lambda x: w2v_model[str(x)]).apply(pd.Series)
        train_features = pd.concat([train_features, embedding_df], axis=1).drop(column_name, axis=1)
        print(column_name, "FINISH")
    train_features.to_csv("main_features.csv", index=False)
    print("FINISH save csv!")

In [ ]:
train_df = pd.read_csv("embed/train/train_embedding_all_1.csv", header=None)

In [ ]:
train_df.head()

In [ ]:
train_np = np.loadtxt("embed/train/train_embedding_all_1.csv", delimiter=", ")
train_np.shape

In [ ]:
train_root = "dataset/train/"
train_user_path = os.path.join(train_root, "user.csv")
train_user = pd.read_csv(train_user_path, index_col="user_id")
train_user.head()
train_user.shape

In [ ]:
train_user_id = train_np[:, 0]
uid = train_user_id.astype(int)
train_age = train_user.loc[uid, "age"]
train_gender = train_user.loc[uid, "gender"]

In [ ]:
train_features = train_np[:, 1:]
train_age = train_age.values - 1
train_gender = train_gender.values - 1

In [ ]:
train_features, valid_features,\
train_age, valid_age,\
train_gender, valid_gender = train_test_split(train_features, train_age, train_gender, test_size=0.33, random_state=42)

In [ ]:
(train_np == 0).sum()

In [ ]:
# (train_np[:, 401:601] == 0).sum()
# (train_np[:, 701:801] == 0).sum()
train_np[train_np == 0] = np.nan

In [ ]:
lgb_traindata_gender = lgb.Dataset(train_features, train_gender)
lgb_traindata_age = lgb.Dataset(train_features, train_age)

lgb_valdata_gender = lgb.Dataset(valid_features, valid_gender, reference=lgb_traindata_gender)
lgb_valdata_age = lgb.Dataset(valid_features, valid_age, reference=lgb_traindata_age)

# 性别模型的预测

In [ ]:
gender_model = lgb_model(model_kind="gender")
gender_model.train(lgb_traindata_gender, lgb_valdata_gender)
gender_model.save_model()

In [ ]:
test_record = pd.merge(test_click, test_ad, on="creative_id")

In [ ]:
test_features = test_record.iloc[:, [0, 2, 3, 4, 5, 6, 7, 8]]
test_features = test_features.values

In [ ]:
test_pred = gender_model.predict(test_features)

In [ ]:
test_pred

In [ ]:
test_np = np.loadtxt("embed/test/test_embedding_all_1.csv", delimiter=", ")
test_np.shape

In [ ]:
cd tencent_algo_2020/

In [ ]:
import data
import lightgbm as lgb
import numpy as np
import os
import sys
import re
import pandas as pd
import matplotlib
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import mean_squared_error, roc_auc_score, accuracy_score
from gensim.models import word2vec, keyedvectors
import logging

from model import lgb_model

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


print("START loading train embedding and train user info")
train_np = np.loadtxt("embed/train/train_embedding_all_1.csv", delimiter=", ")
train_np[train_np == 0] = np.nan

train_root = "dataset/train/"
train_user_path = os.path.join(train_root, "user.csv")
train_user = pd.read_csv(train_user_path, index_col="user_id")
print("FINISH load train_np, train_user")
print("===========================================================================")

print("START get train_features, train_age, train_gender, and random split train/valid data")
uid = train_np[:, 0].astype(int)
train_age = train_user.loc[uid, "age"]
train_gender = train_user.loc[uid, "gender"]

train_features = train_np[:, 1:]
train_age = train_age.values - 1
train_gender = train_gender.values - 1

train_features, valid_features,\
train_age, valid_age,\
train_gender, valid_gender = train_test_split(train_features,\
                                              train_age,\
                                              train_gender,\
                                              test_size=0.33,\
                                              random_state=42)
print("FINISH random split train/valid data")
print("===========================================================================")

print("START construct lgb train valid data")
lgb_traindata_gender = lgb.Dataset(train_features, train_gender)
lgb_traindata_age = lgb.Dataset(train_features, train_age)

lgb_valdata_gender = lgb.Dataset(valid_features, valid_gender, reference=lgb_traindata_gender)
lgb_valdata_age = lgb.Dataset(valid_features, valid_age, reference=lgb_traindata_age)
print("FINISH construct lgb train valid data")
print("===========================================================================")


In [ ]:
# 导入已保存模型
gender_model = lgb_model(model_kind="gender")
gender_model.load_model()
age_model = lgb_model(model_kind="age")
age_model.load_model()

In [ ]:
print("START valid acc of predict")
# TODO 性别模型的预测
valid_gender_predict = gender_model.predict(valid_features)
valid_gender_predict = gender_model.transform_pred(valid_gender_predict)
acc_gender = accuracy_score(valid_gender_predict, valid_gender)

# TODO 年龄模型的预测
valid_age_predict = age_model.predict(valid_features)
valid_age_predict = age_model.transform_pred(valid_age_predict)
acc_age = accuracy_score(np.array(valid_age_predict), valid_age)

print("In valid data, accuracy of gender is {}, accuracy of age is {}".format(acc_gender, acc_age))
print("FINISH")
print("===========================================================================")

In [ ]:
print("START test predict")
test_np = np.loadtxt("embed/test/test_embedding_all_1.csv", delimiter=", ")
test_np[test_np == 0] = np.nan
test_uid = test_np[:, 0].astype(int)
test_features = test_np[:, 1:]

# TODO 性别模型的预测
test_gender_predict = gender_model.predict(test_features)
test_gender_predict = gender_model.transform_pred(test_gender_predict)
# TODO 年龄模型的预测
test_age_predict = age_model.predict(test_features)
test_age_predict = age_model.transform_pred(test_age_predict)

result = pd.DataFrame({"user_id": test_uid, "predicted_age": test_age_predict, "predicted_gender": test_gender_predict})
result.to_csv("results.csv", index=False)

print("FINISH ALL and save result to results.csv")
print("===========================================================================")

In [ ]:
train_ad, train_click, train_user, test_ad, test_click = data.load_data()
# train_record
train_record = pd.merge(train_click, train_ad, on="creative_id")
# test_record
test_record = pd.merge(test_click, test_ad, on="creative_id")

In [ ]:
grouped = test_record.groupby("user_id")

In [ ]:
test_np = np.loadtxt("embed/test/test_embedding_all_1.csv", delimiter=", ")

In [ ]:
for user_id, record in grouped:
    print(user_id)
    print(record)
    break